# PyUIT GUI Tools Demo

PyUIT provides a module called `gui_tools` that provides some basic building blocks for creating web-based graphical interfaces that interact with the HPC. The `gui_tools` leverage a library called [Panel](https://panel.holoviz.org/).

## Configuration

If you haven't already configured client in UIT+ then first see the `PyUIT Demo` notebook for instructions on configuration.

In [ ]:
from uit import Client
from uit.gui_tools import HpcAuthenticate, HpcConnect, HpcSubmit, HpcJobMonitor

import panel as pn
pn.extension()  # In order for Panel or render widgets in a Notebook, you need to run this.

## GUI Building Blocks

Panel uses something called a [Pipline](https://panel.holoviz.org/user_guide/Pipelines.html) to create a workflow or series of progressive steps in a user interface. Each step in the pipline is called a stage. PyUIT creates some basic stages needed to create a workflow for submitting jobs to the HPC. Some of these stages are shwon below.

### Authenticate Stage

This stage doesn't have any direct interface but handles re-directing the user to the UIT+ authentication page and handles retreiving the token to authorize the user to make further calls to the HPC.

In [ ]:
auth = HpcAuthenticate()
auth.panel()

### Connect Stage

The connect stage allows the user to specify an HPC system to connect to, and even provides some advanced tools for controlling the specific login node to connect to.

In [ ]:
conn = HpcConnect(uit_client=auth.uit_client)
conn.panel()

## Submit Stage

The submit stage provides widgets for setting up a PBS script and submitting it to the HPC job queuing system.

In [ ]:
sub = HpcSubmit(uit_client=conn.uit_client)
sub.panel()

## Creating Custom Stages

PyUIT provides the basic building blocks for a pipeline, but to create a full pipeline that actually submits a job to the HCP you will need to customize some of these stages and possibly create new ones. Below is an example of setting up some new/customized stages. For more information about creating pipline stages refer to the [pipline documentation](https://panel.holoviz.org/user_guide/Pipelines.html).

In [ ]:
import param

class HpcIntro(param.Parameterized):
    title = 'PyUIT Pipeline Intro'
    start_new_btn = param.Action(lambda self: self.next(), label='Start New Job')
    ready = param.Boolean(default=False, precedence=-1)

    def next(self):
        self.ready = True

    def panel(self):
        title = pn.pane.HTML(f'<h1>{self.title}</h1>', width=350)

        return pn.Column(
            title,
            pn.pane.Str(
                f'A Pipeline strings several stages together in sequence. Click "{self.param.start_new_btn.label}" to begin a new job. '
                'You will be redirected in a new tab to authenticate to the HPC. Once you have authenticated return to this tab to continue the pipeline.',
                sizing_mode='stretch_width',
                width=500,
            ),
            pn.Row(
                pn.Param(self.param.start_new_btn, widgets={'start_new_btn': {'button_type': 'success'}}),
            ),
        )
    
class HpcJobScriptEditor(param.Parameterized):
    uit_client = param.ClassSelector(Client)
    file_type = param.ObjectSelector(default='sh', objects=['py', 'sh'])
    file_contents = param.String(default='echo Hello World!')
    ready = param.Boolean(default=False, precedence=-1)
    continue_btn = param.Action(lambda self: self.next(), label='Continue')
    
    def next(self):
        self.ready = True
    
    @param.output(execution_block_contents=str)
    def output(self):
        return self.file_contents
    
    @param.depends('file_type')
    def editor(self):
        file_type = pn.Param(self.param.file_type)
        editor = pn.widgets.Ace.from_param(self.param.file_contents, language=self.file_type, sizing_mode='stretch_width')
        editor.jslink(file_type, {'language': 'value'})
        return pn.Column(file_type, editor, sizing_mode='stretch_width')
    
    def panel(self):
        return pn.Column(
            '# Create Execution Script',
            self.editor,
            pn.widgets.Button.from_param(self.param.continue_btn, button_type='success', width=200),
            sizing_mode='stretch_width',
        )
    
class HpcSubmitScript(HpcSubmit):
    execution_block_contents = param.String()
    
    @property
    def execution_block(self):
        return self.execution_block_contents

## Creating a Pipeline

A pipline is just a sequence of stages. Below is an example of how to string all of the stages together to create a pipline.

In [ ]:
p = pn.pipeline.Pipeline(auto_advance=True, ready_parameter='ready', debug=True)

p.add_stage('Intro', HpcIntro)
p.add_stage('Authenticate', HpcAuthenticate)
p.add_stage('Connect', HpcConnect)
p.add_stage('Inputs', HpcJobScriptEditor)
p.add_stage('Submit', HpcSubmitScript)
p.add_stage('Monitor', HpcJobMonitor)

p.layout

## Launching a GUI outside of Jupyter

Panel lets you easily run any of it's GUI objects both in and out of Jupyter. To serve any Panel object outside of Jupyter just add a `.show()` to the end of the object.

In [ ]:
p.layout.show()

## Other GUI Tools

In addition to stages that are ment to be part of a pipline, PyUIT also provides some other GUI building blocks that can be used when creating custom stages. Some of these are shown below.

### FileBrowser and HPCFileBrowser

A `FileBrowser` object allows the user to browse the local file system. The `HPCFileBrowser` similarly allows the user to browse the HPC file system, and requires an authenticated `Client` in order to make calls to the HPC.

In [ ]:
from uit.gui_tools import FileBrowser, HpcFileBrowser

In [ ]:
file_browser = FileBrowser()
file_browser.panel()

In [ ]:
uit_client = auth.uit_client  # Note, that you must run the Authenticate Stage cell above to get an authenticated client

hpc_file_browser = HpcFileBrowser(uit_client=uit_client)
hpc_file_browser.panel()

### FileSelector

Perhaps more useful than just browsing the file system is the ability to select a file. The `FileSelector` uses a file browser but allows the user to select files.

In [ ]:
from uit.gui_tools import FileSelector

In [ ]:
file_selector = FileSelector(file_browser=hpc_file_browser)
file_selector.panel

In [ ]:
file_selector.file_path

### FileViewer

The `FileViewer` uses a `FileSelector` and then renders the file that the user selects.

In [ ]:
from uit.gui_tools import FileViewer

In [ ]:
file_viewer = FileViewer(file_select=file_selector, uit_client=uit_client)
file_viewer.panel()